In [ ]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath('../code'))
import config
from spellcheck import Spellcheck
from util import Util

# hny_pap_nl = pd.read_csv("../data/hny/pap-nl.csv", na_filter=False)
nbo_pap = pd.read_csv("../data/nbo/pap(cap).csv", na_filter=False)
corpus = Util.attachType(nbo_pap, "pap-simple")
corpus = corpus[corpus["type"]=="word"]
words = corpus["pap-simple"].values
spell = Spellcheck(spellchecker_corpus=words)

sentence = "mi man no ta bon pero manan mi kuna ta trese anto bini bek"
big_sentence = ". Presidente di Banko Sentral di Kòrsou i St. Maarten sr. Richard Doornbosch... WILLEMSTAD.- Buskando solushon fo’i 2013 te ku 2019 pa Girobank, a hasi investigashon di e banko. A aserka APC i IIG TOF Holding N.V. (ámbos ta doño di Girobank ku respektivamente 42% i 58%) pa un plan di restrukturashon i re-kapitalisashon pa tin sufisiente kapital den e banko; identifiká potensial kumpradónan ku ke a invertí den e banko. Esaki no tabata algu fásil. Tabatin un défisit"

In [ ]:
import random
corpus_len = nbo_pap.copy()
corpus_len["len"] = nbo_pap.apply(lambda row: len(row["pap-simple"]), axis=1)


random.seed(10)
def screwWord(word):
    screwed = word
    
    word_length = len(word)
    for i in range(0, word_length):
        screwed = screwed[:i] + screwed[random.randrange(0, word_length)] + screwed[i+1:]
    return screwed


# for word in four_raw:
corpus_len["w"] = corpus_len.apply(lambda row: screwWord(row["pap-simple"]), axis=1)
four_raw = corpus_len[corpus_len.len == 4]
test_four = four_raw.head(10)["w"].values

In [ ]:
# Translate.getWordCorrections(sentence, corpus)

In [ ]:
# trans.getFastWordCorrections(sentence, check_alternatives=True)

In [ ]:
import nltk
# trans.getMixedWordCorrections(sentence, corpus)
smarter = "iasabra el a bisami ekos kune i use"
# input_words = Util.findWords(big_sentence)
input_words = test_four
# print(input_words)

In [ ]:
spell.getMixedWordCorrections(input_words, corpus, "Levenstein")

In [ ]:
spell.getMixedWordCorrections(input_words, corpus, metric="Hamming")

In [ ]:
spell.getWordCorrections(input_words, metric="Levenstein")

In [ ]:
def extremeFind(self, word, d):
        found_word = "" 
        current = self
        print(f"word:{word} depth:{d}")
        for i, l in enumerate(word):

            print(f"index {i}")
            found_key = False
            # print(f"checking if {l} has accented")
            if d > 0 and l in current.children:
                current = current.children[l]
                found_word += l
                print(f"Appending letter {l}")
                found_key = True
            elif d == 0:
                print(f"items are len {len(current.children.keys())}")

                for key, child in current.children.items():
                    # print(f"checking {accented_l}")
                    # print(f"found {accented_l}")
                    rest_search = word[i+1:]

                    print(f"Word {word}, key {key}, ind {i}, d {d}")
                    print(f"Checking {rest_search}")
                    found_rest_word = extremeFind(child, rest_search, d=d+1)
                    print(f"Found {found_rest_word}")
                    if found_rest_word:
                        return key + found_rest_word
                

            if not found_key:
                return ""
            
            if i == len(word)-1:
                if current.lastNode:
                    return found_word
                else:
                    return ""
def getWordCorrections2(words):
        translations = {}   
        for word in words:
            # Make sure to ignore case if you're making word lowercase
            exists = spell.trie.find(word)
            if not exists:
                all_matches = extremeFind(spell.trie, word, d=0)
                # matches = all_matches[:3]
                return all_matches
                # print(word, matches)
                # if matches and matches[0][1] > 0:
                if matches:
                    # print(word, words_corpus.head(3)["closest"].to_list())
                    translations[word] = list(map(lambda x: x[0], matches))
                elif not matches:
                    translations[word] = ["This word is incorrect"]
                
        return translations
# getWordCorrections2(input_words)
getWordCorrections2(["iopi"])


In [ ]:
# characters_test = "(whatever)i don't know, but random. 15 a)e right {,age}()((er)"
characters_test = "min sa 42% 32 fo'i ki dia re-kapitulashon M'a mire n' ta nothing' bai fo'i 'esaki tambe' 'gani sa?' e"
# characters_test = "Buskando solushon fo’i 2013 te ku 2019"
import re
def getWords(sentence):
    # words = re.findall(r'^(.*?)[;\.,!\(].*',without_num)        
    # words = re.findall(r'[^\w\d\'](\w+)[^\w\d\']',sentence) 
    # words = re.findall(r'\b([^\d\W]+[\'-]?[^\d\W\']*)\b',sentence) 
    words = re.findall(r'\b([^\d\W]+[\'\’-]*[^\d\W\']*)',sentence) 
    # words = re.findall(r'\b([\d\w\'-]+)\b',sentence) 
    # words = re.findall(r'([a-z]+[\'-]?(?![\w\'-])[a-z]*)',sentence) 
    return words
getWords(characters_test)
# getWords(big_sentence)